# Causal Analysis of Carnatic Music

This notebook  primarily contains the code featured in the article with the same name,
published under the banner of IEEE **Proceedings of CONECCT 2021**.

This notebook defines a basic protocol that can be followed to achieve the results featured
in the published article.

**Warning: The code shown here for causal analysis is outdated. Refer `raga_surrogate_causality.ipynb` for the updated code**

The deprecated files `raga_exps_causality.ipynb`, `raga_exps_causality2.ipynb`,
`raga_exps_causality_work.ipynb` and `raga_exps_causality_work65.ipynb`, along with this one,
all follow the same logic:

- <u>*Step 1: Data Extraction*</u>: Parsed notations are extracted from the respective file in `dataset.BASE_PATH` (`dataset` is one of the import scripts). The files under `dataset.BASE_PATH` that are relevant are saved as `<ragaId>/<composition_name>-parsed.txt`
- <u>*Step 2: Data Processing*</u>: The extracted data is then converted to it's corresponding adjusted melody, i.e. the pitch indices are repeated a particular number of times proportional to the note-event duration,
- <u>*Step 3: Causal Analysis*</u>: The adjusted melodies are then chosen pairwise, and every pair is used as an argument for `ETC.CCM_Causality()`. In this manner, causality is inferred for each pair.
- <u>*Step 4: Result Visualization*</u>: Using the results above, the utility functions in `draw.py` are employed to "print" the final result. A valid **GraphViz** code is output, which can be verified by running the same on [Graphviz Online](https://dreampuf.github.io/GraphvizOnline)

In [1]:
#import dependencies
import data
import time
from utils import dataset, distances, draw, parallelizer, plotter, dataStructures
import numpy as np

To follow the general algorithm above, a method can be defined. The `print_correct()` procedure
defined below does exactly that. It returns a Tuple (correct_connections, total_connections, connection_strengths),
wherein, correct_connections (`tc`) is the total number of connections where the *Melakarta* composition points towards (or "causes")
the _Janya_ compositions. total_connections (`totc`) is the total number of connections possible between all the
*Melakarta* and *Janya* compositions. In other words,

\begin{aligned}
total\_connections = comps_{Mela} \times comps_{Janya},
\end{aligned}

where, $comps_{Mela}$ is the total number of *Melakarta* compositions in the set, and
$comps_{Janya}$ is the total number of *Janya* compositions in the set.

Therefore, total number of pairs to be considered (without repetition) is:

\begin{aligned}
n_{pairs} = ^{total\_connections}C_2
\end{aligned}

Hence, connection_strengths (`strengths`) is a list of $n_{pairs}$ elements, wherein, each
element represents the strength of causality detected in the corresponding pair

In [2]:
def print_correct(mela_id='22', janya_id='22_a', time_cal=None):
    # Data Extraction
    dat = dataset.GetRagaSongCoordsConcat(mela_id, janya_id)
    if time_cal is not None:
        assert isinstance(time_cal, dict)
    for key in dat:
        dat[key] = dataStructures.PackTuples(*dat[key])

    # Data Processing
    adj = dataset.GetAdjustedMelodies(dat)
    print("finding causality....")
    t3 = time.perf_counter()

    # Causal Inference
    tc, totc, strengths = parallelizer.TrueLZPCausality_listParallel(list(adj.values()), list(adj.keys()), mela=dataset.GetRagaFromRagaId(mela_id))
    t4 = time.perf_counter()

    if time_cal is not None:
        time_cal["Causality{}".format(mela_id)] = t4 - t3
    return tc, totc, strengths

The `print_correct()` procedure is expected to return a different outcome every time. The
`ragaIterator()` method repeats `print_correct()` `n_iters` times. It returns a Tuple
of lists `(tcs, totcs, strengths)`, all of lengths `n_iters`. As explained above, `tcs` is
the list of correct connections in each iteration, `totcs` is made up of constant elements, and
is being kept there for brevity, and `strengths` is made up of arrays of causal strengths in each iteration.

In [3]:
def ragaIterator(n_iters, mela, janya, times_it=None):
    if times_it is not None:
        assert isinstance(times_it, dict)
        times_it["Causality{}".format(mela)] = []
    tcs = []
    strengths = []
    totcs = []
    t1 = time.perf_counter()
    for i in range(n_iters):
        print("iteration: ", i + 1)
        tm = {}
        tc, totc, st = print_correct(mela, janya, time_cal=tm)
        if times_it is not None:
            times_it["Causality{}".format(mela)].append(tm["Causality{}".format(mela)])
        t = time.perf_counter()
        print("time elapsed: ", (t - t1))
        t1 = t
        tcs.append(tc)
        totcs.append(totc)
        strengths.append(st)
    return tcs, totcs, strengths

A lambda function to calculate the average of a given array/list

In [4]:
avg_conns = lambda t_c : np.array(t_c).mean()

In [5]:
t_dict = {}

## Causal Analysis of *raga* in Carnatic Music

1. _Raga Kalyani - Raga Hamsadhwani_

In [6]:
tc65, totc65, ss65 = ragaIterator(50, '65', '29_h', times_it=t_dict)

iteration:  1
finding causality....
21 true causes out of 24 total connections
time elapsed:  5.458329000975937
iteration:  2
finding causality....
18 true causes out of 24 total connections
time elapsed:  5.249777334975079
iteration:  3
finding causality....
16 true causes out of 24 total connections
time elapsed:  5.293846443062648
iteration:  4
finding causality....
16 true causes out of 24 total connections
time elapsed:  5.337825160007924
iteration:  5
finding causality....
20 true causes out of 24 total connections
time elapsed:  5.330514708999544
iteration:  6
finding causality....
21 true causes out of 24 total connections
time elapsed:  5.286794876912609
iteration:  7
finding causality....
18 true causes out of 24 total connections
time elapsed:  5.417187775019556
iteration:  8
finding causality....
19 true causes out of 24 total connections
time elapsed:  5.48275459499564
iteration:  9
finding causality....
18 true causes out of 24 total connections
time elapsed:  5.613216281

In [7]:
avg_conns(tc65), avg_conns(t_dict["Causality65"])

(18.54, 4.391181174484082)

2. _Raga Mayamalavagowla-Raga Malahari_

In [6]:
tc15, totc15, ss15 = ragaIterator(50, '15', '15_m', times_it=t_dict)

iteration:  1
finding causality....
24 true causes out of 24 total connections
time elapsed:  2.107289146995754
iteration:  2
finding causality....
24 true causes out of 24 total connections
time elapsed:  2.1197836920036934
iteration:  3
finding causality....
24 true causes out of 24 total connections
time elapsed:  2.1665766980004264
iteration:  4
finding causality....
23 true causes out of 24 total connections
time elapsed:  2.141128106988617
iteration:  5
finding causality....
24 true causes out of 24 total connections
time elapsed:  2.0807854150043568
iteration:  6
finding causality....
24 true causes out of 24 total connections
time elapsed:  2.0737881819950417
iteration:  7
finding causality....
23 true causes out of 24 total connections
time elapsed:  2.1345080690080067
iteration:  8
finding causality....
23 true causes out of 24 total connections
time elapsed:  2.1418956559937214
iteration:  9
finding causality....
23 true causes out of 24 total connections
time elapsed:  2.14

In [7]:
avg_conns(tc15), avg_conns(t_dict["Causality15"])

(23.38, 1.3697117266608985)

3. _Raga Hanumatodi - Raga Dhanyasi_

In [8]:
tc8, totc8, ss8 = ragaIterator(50, '8', '8_d', times_it=t_dict)

iteration:  1
finding causality....
10 true causes out of 48 total connections
time elapsed:  7.093012323995936
iteration:  2
finding causality....
14 true causes out of 48 total connections
time elapsed:  7.278605077997781
iteration:  3
finding causality....
11 true causes out of 48 total connections
time elapsed:  7.1360680110083194
iteration:  4
finding causality....
7 true causes out of 48 total connections
time elapsed:  7.19748635799624
iteration:  5
finding causality....
16 true causes out of 48 total connections
time elapsed:  7.2402704959968105
iteration:  6
finding causality....
11 true causes out of 48 total connections
time elapsed:  7.2110268760006875
iteration:  7
finding causality....
16 true causes out of 48 total connections
time elapsed:  7.280427276011324
iteration:  8
finding causality....
9 true causes out of 48 total connections
time elapsed:  7.14904898799432
iteration:  9
finding causality....
12 true causes out of 48 total connections
time elapsed:  7.225316287

In [9]:
avg_conns(tc8), avg_conns(t_dict["Causality8"])

(10.4, 5.6361683101599915)

4. _Raga Karaharapriya - Raga Aabhogi_

In [10]:
tc22, totc22, ss22 = ragaIterator(50, '22', '22_a', times_it=t_dict)

iteration:  1
finding causality....
18 true causes out of 18 total connections
time elapsed:  5.35680692299502
iteration:  2
finding causality....
16 true causes out of 18 total connections
time elapsed:  5.410991673998069
iteration:  3
finding causality....
14 true causes out of 18 total connections
time elapsed:  5.215870358006214
iteration:  4
finding causality....
17 true causes out of 18 total connections
time elapsed:  5.355977818995598
iteration:  5
finding causality....
18 true causes out of 18 total connections
time elapsed:  5.338109313001041
iteration:  6
finding causality....
16 true causes out of 18 total connections
time elapsed:  5.33044441700622
iteration:  7
finding causality....
16 true causes out of 18 total connections
time elapsed:  5.191618240001844
iteration:  8
finding causality....
18 true causes out of 18 total connections
time elapsed:  5.532466545992065
iteration:  9
finding causality....
16 true causes out of 18 total connections
time elapsed:  5.1868477430

In [11]:
avg_conns(tc22), avg_conns(t_dict["Causality22"])

(16.72, 4.111615597240452)

5. _Raga Harikambhoji - Raga Kambhoji_

In [12]:
tc28, totc28, ss28 = ragaIterator(50, '28', '28_k', times_it=t_dict)

iteration:  1
finding causality....
5 true causes out of 20 total connections
time elapsed:  4.6078283060051035
iteration:  2
finding causality....
4 true causes out of 20 total connections
time elapsed:  4.766021547999117
iteration:  3
finding causality....
9 true causes out of 20 total connections
time elapsed:  4.743822279997403
iteration:  4
finding causality....
2 true causes out of 20 total connections
time elapsed:  4.707711499999277
iteration:  5
finding causality....
5 true causes out of 20 total connections
time elapsed:  4.729427800994017
iteration:  6
finding causality....
1 true causes out of 20 total connections
time elapsed:  4.56856753800821
iteration:  7
finding causality....
8 true causes out of 20 total connections
time elapsed:  4.72145724100119
iteration:  8
finding causality....
0 true causes out of 20 total connections
time elapsed:  4.647320828997181
iteration:  9
finding causality....
7 true causes out of 20 total connections
time elapsed:  4.6196432069991715
i

In [13]:
avg_conns(tc28), avg_conns(t_dict["Causality28"])

(4.48, 3.472000266239338)

6. _Raga Shakarabharanam - Raga Hamsadhwanni_

In [6]:
tc29, totc29, ss29 = ragaIterator(50, '29', '29_h', times_it=t_dict)

iteration:  1


TypeError: '<' not supported between instances of 'int' and 'NoneType'

In [15]:
avg_conns(tc29), avg_conns(t_dict["Causality29"])

(21.58, 3.7903087885395506)

## Visualization as DAG(s)

1. _Raga Karaharapriya - Raga Aabhogi_

In [2]:
# Data Extraction and Processing
dict_adjMel = dataset.GetAdjMelDictsConcat('22', '22_a')
print(list(dict_adjMel.keys()))
# parallelizer.TrueLZPCausality_listParallel(list(dict_adjMel.values()), list(dict_adjMel.keys()), mela=dataset.GetRagaFromRagaId('22'))
# Causal Inference (Parallelized)
dists = parallelizer.list_parallelizer(distances.GetLZPCausality, list(dict_adjMel.values()))
# Visualization
draw.draw_causal_inference(dists, list(dict_adjMel.keys()), mela=dataset.GetRagaFromRagaId('22'))


Final Value Chosen: 112240
['janaki-pate(Kharaharpriya)', 'chakkani-raja(Kharaharpriya)', 'pakkala-nilabadi(Kharaharpriya)', 'rama-nee-samana(Kharaharpriya)', 'paraamukham(Kharaharpriya)', 'okapari(Kharaharpriya)', 'sabaapatikku(Abhogi)', 'evvariBodhana(Abhogi)', 'srilakshmivaraham(Abhogi)']
digraph "" {
	graph [bb="0,0,4549.8,413.47",
		rankdir=TD
	];
	node [label="\N"];
	"janaki-pate(Kharaharpriya)"	[color=black,
		fontcolor=white,
		fontsize=42,
		height=1.0607,
		pos="421.44,375.29",
		style=filled,
		width=11.707];
	"sabaapatikku(Abhogi)"	[fontsize=42,
		height=1.0607,
		pos="1688.4,262.92",
		width=9.487];
	"janaki-pate(Kharaharpriya)" -> "sabaapatikku(Abhogi)"	[pos="e,1423.4,287.01 719.69,348.31 929.68,330.01 1208.7,305.71 1413.4,287.88",
		weight=1.0];
	"evvariBodhana(Abhogi)"	[fontsize=42,
		height=1.0607,
		pos="2668.4,38.184",
		width=10.174];
	"janaki-pate(Kharaharpriya)" -> "evvariBodhana(Abhogi)"	[pos="e,2324.6,51.404 528.46,338.29 708.13,279.6 1082.9,164.6 1409.4,112.37 

2. _Raga Harikambhoji - Raga Kambhoji_

In [3]:
dict_adjMel = dataset.GetAdjMelDictsConcat('28', '28_k')
print(dict_adjMel.keys())

dists = parallelizer.list_parallelizer(distances.GetLZPCausality, list(dict_adjMel.values()))
draw.draw_causal_inference(dists, list(dict_adjMel.keys()), mela=dataset.GetRagaFromRagaId('28'))

Final Value Chosen: 98556
dict_keys(['entaRaniTanakenta(Harikambodhi)', 'dinamaniVamsha(Harikambodhi)', 'ramaNannu(Harikambodhi)', 'endukuNidraya(Harikambodhi)', 'oRangashayi(Kambodhi)', 'ratnaKanchuka(Kambodhi)', 'kamalaambakaayi(Kambodhi)', 'ivanYaro(Kambodhi)', 'evarimata(Kambodhi)'])
digraph "" {
	graph [bb="0,0,2098.4,638.21",
		rankdir=TD
	];
	node [label="\N"];
	"entaRaniTanakenta(Harikambodhi)"	[color=black,
		fontcolor=white,
		fontsize=42,
		height=1.0607,
		pos="1365.4,38.184",
		style=filled,
		width=15.065];
	"dinamaniVamsha(Harikambodhi)"	[color=black,
		fontcolor=white,
		fontsize=42,
		height=1.0607,
		pos="1555.4,487.65",
		style=filled,
		width=14.064];
	"ratnaKanchuka(Kambodhi)"	[fontsize=42,
		height=1.0607,
		pos="1606.4,375.29",
		width=11.589];
	"dinamaniVamsha(Harikambodhi)" -> "ratnaKanchuka(Kambodhi)"	[pos="e,1589.2,413.61 1572.7,449.29 1576.6,440.75 1580.9,431.57 1585,422.7",
		weight=1.0];
	"kamalaambakaayi(Kambodhi)"	[fontsize=42,
		height=1.0607,
		pos="86

3. _Raga Hanumatodi - Raga Dhanyasi_

In [4]:
dict_adjMel = dataset.GetAdjMelDictsConcat('8', '8_d')
print(dict_adjMel.keys())

dists = parallelizer.list_parallelizer(distances.GetLZPCausality, list(dict_adjMel.values()))
draw.draw_causal_inference(dists, list(dict_adjMel.keys()), mela=dataset.GetRagaFromRagaId('8'))

Final Value Chosen: 84960
dict_keys(['kaddanuvariki(Hanumatodi)', 'rave-himagiri-kumari(Hanumatodi)', 'ninnu-vina(Hanumatodi)', 'era-napai(Hanumatodi)', 'gajavadana(Hanumatodi)', 'nivanti-deviamunu(Hanumatodi)', 'koluvamare-gada(Hanumatodi)', 'kamalaambike(Hanumatodi)', 'bhavamulona(Dhanyaasi)', 'balakrishnan(Dhanyaasi)', 'sangeetha-gyanamanu(Dhanyaasi)', 'paradevata(Dhanyaasi)', 'rama-daya(Dhanyaasi)', 'kizh-vanam(Dhanyaasi)'])
digraph "" {
	graph [bb="0,0,4978.4,638.21",
		rankdir=TD
	];
	node [label="\N"];
	"kaddanuvariki(Hanumatodi)"	[color=black,
		fontcolor=white,
		fontsize=42,
		height=1.0607,
		pos="4362.4,262.92",
		style=filled,
		width=12.119];
	"rama-daya(Dhanyaasi)"	[fontsize=42,
		height=1.0607,
		pos="2675.4,150.55",
		width=9.7227];
	"kaddanuvariki(Hanumatodi)" -> "rama-daya(Dhanyaasi)"	[pos="e,2987.8,167.84 4058.4,235.52 4011.2,231.71 3963,227.98 3917.4,224.74 3605.2,202.5 3248.6,181.99 2997.8,168.38",
		weight=1.0];
	"rave-himagiri-kumari(Hanumatodi)"	[color=black,
	

In [8]:
dict_adjMel = dataset.GetAdjMelDictsConcat('65', '29_h')
print(dict_adjMel.keys())

# lists = []
# for val in dict_adjMel.values():
#     lists.append(list(val))

# dists = distances.GetCausalityMetrics(lists)[0]
#
dists = parallelizer.list_parallelizer(distances.GetLZPCausality, list(dict_adjMel.values()))
draw.draw_causal_inference(dists, list(dict_adjMel.keys()), mela=dataset.GetRagaFromRagaId('65'), janya=dataset.GetRagaFromRagaId('29_h'))

Final Value Chosen: 114620
dict_keys(['Nidhicala(Kalyani)', 'Pankajalocana(Kalyani)', 'Sivakaameswari(Kalyani)', 'Kamalambam(Kalyani)', 'Himadrisute(Kalyani)', 'Sive-Pahimam(Kalyani)', 'Jalajakshi(Hamsadhwani)', 'Vatapi(Hamsadhwani)', 'Raghunayaka(Hamsadhwani)', 'Vaarana(Hamsadhwani)'])
digraph "" {
	graph [bb="0,0,1518.8,638.21",
		rankdir=TD
	];
	node [label="\N"];
	Nidhicala	[color=black,
		fontcolor=white,
		fontsize=42,
		height=1.0607,
		pos="1049.7,375.29",
		style=filled,
		width=4.3408];
	Jalajakshi	[fontsize=42,
		height=1.0607,
		pos="499.68,38.184",
		width=4.3408];
	Nidhicala -> Jalajakshi	[pos="e,639.69,55.399 1029.4,337.29 992.99,272.18 916.26,141.24 872.68,112.37 836.06,88.108 736.67,69.29 649.69,56.813",
		weight=1.0];
	Raghunayaka	[fontsize=42,
		height=1.0607,
		pos="621.68,262.92",
		width=5.9908];
	Nidhicala -> Raghunayaka	[pos="e,741.13,294.72 944.16,347.08 885.77,332.02 812.74,313.19 750.94,297.25",
		weight=1.0];
	Vaarana	[fontsize=42,
		height=1.0607,
		pos="11

In [ ]:
# dict_adjMel = dataset.GetAdjMelDictsConcat('29', '29_h', unif=False)
# print(dict_adjMel.keys())
#
# lists = []
# for val in dict_adjMel.values():
#     lists.append(list(val))
#
# dists = distances.GetCausalityMetrics(lists)[0]
# # dists = parallelizer.list_parallelizer(distances.GetLZPCausality, list(dict_adjMel.values()))
# draw.draw_causal_inference(dists, list(dict_adjMel.keys()), mela=dataset.GetRagaFromRagaId('29'), janya=dataset.GetRagaFromRagaId('29_h'))
#
# dict_adjMel = dataset.GetAdjMelDictsConcat('65', '29_h', unif=False)
# print(dict_adjMel.keys())
#
# lists = []
# for val in dict_adjMel.values():
#     lists.append(list(val))
#
# dists = distances.GetCausalityMetrics(lists)[0]
#
# # dists = parallelizer.list_parallelizer(distances.GetLZPCausality, list(dict_adjMel.values()))
# draw.draw_causal_inference(dists, list(dict_adjMel.keys()), mela=dataset.GetRagaFromRagaId('65'), janya=dataset.GetRagaFromRagaId('29_h'))
#

# Other Experiments

Until now, this notebook portrayed a way for Causal Analysis of Carnatic Music. Further
down, other tools, which might be useful for experimentation with distance metrics (e.g. ones
defined in `distances.py`) are shown. One such tool is the 2D representation of compositions.

Causal Analysis uses the 3D representation of the compositions. The 2D representation of
composition is depicted below:

In [2]:
dict_coords = dataset.GetRagaSongCoords2d('8')
dict_coords

{'kamalaambike(Hanumatodi)': [(0.5, 10),
  (1.0, 99999),
  (1.5, 10),
  (2.0, 99999),
  (2.5, 8),
  (2.75, 10),
  (3.0, 7),
  (3.5, 8),
  (4.0, 99999),
  (4.25, 8),
  (4.5, 10),
  (5.0, 12),
  (5.5, 12),
  (6.0, 10),
  (1, 0),
  (1.5, 99999),
  (2.0, 99999),
  (2.5, 99999),
  (3.0, -4),
  (3.25, 99999),
  (3.5, -2),
  (4.0, 0),
  (4.25, 99999),
  (4.5, -2),
  (5.0, -2),
  (5.5, -4),
  (6.0, -5),
  (6.5, -7),
  (2, 10),
  (2.5, 99999),
  (3.0, 10),
  (3.5, 99999),
  (3.75, 8),
  (4.0, 10),
  (4.25, 12),
  (4.5, 13),
  (4.75, 13),
  (5.0, 12),
  (5.25, 10),
  (5.5, 8),
  (5.75, 8),
  (6.0, 10),
  (6.5, 12),
  (7.0, 12),
  (7.5, 10),
  (3, 0),
  (3.461538, 99999),
  (3.923076, 99999),
  (4.384614, 99999),
  (4.615383, -5),
  (4.846152, -4),
  (5.30769, -2),
  (5.538459, -4),
  (5.769228, -2),
  (6.230766, 0),
  (6.461535, -4),
  (6.692304, -2),
  (7.153842, 0),
  (7.384611, 1),
  (7.61538, 0),
  (7.8461490000000005, -2),
  (8.076918000000001, -4),
  (8.307687000000001, -5),
  (8.538456000

Notice the values are made up of Tuples of the form `(x, y)`. Also, notice the progressively
increasing `x` value. `x` here represents the onset of the corresponding `y` on the time-axis.
`y` represents the pitch index.

The following code computes the mean length of composition in _raga Hanumatodi_

In [4]:
sum_lens = 0
for key, val in dict_coords.items():
    print("{} : array length: {}".format(key, len(val)))
    sum_lens += len(val)

print("Mean Lengths: {}".format(sum_lens / len(dict_coords)))

kamalaambike(Hanumatodi) : array length: 608
nivanti-deviamunu(Hanumatodi) : array length: 738
rave-himagiri-kumari(Hanumatodi) : array length: 177
gajavadana(Hanumatodi) : array length: 856
ninnu-vina(Hanumatodi) : array length: 604
koluvamare-gada(Hanumatodi) : array length: 1252
kaddanuvariki(Hanumatodi) : array length: 896
aragimpave(Hanumatodi) : array length: 412
Mean Lengths: 692.875


The following code implements the `player.py` module to save a basic rendition of the
parsed notations in the PCM-s16-le format (16 bit wave or simply `.wav`), at `player.BASE_PATH`.
The sample rate of the output is set to 44.1 kHz. The `note_len` parameter sets the default
length of 1 count in samples (here, 1 count = 22050 samples = 0.5 sec). The _Sa_ (pitch index = 0) is set
to 256.0 Hz, which is the centre frequency for the note C4.

In [3]:
from utils import player

for key in dict_coords:
    p = player.Player(label=key, note_len=22050)
    p.SaveToWAV(dict_coords[key])
    print('{} successfully saved!'.format(key))


event duration: 11025, frequency: 456.1401436878537
11025 11025 0 11024
event duration: 11025, frequency: 0.0
22050 22050 0 22049
event duration: 11025, frequency: 456.1401436878537
33075 33075 0 33074
event duration: 11025, frequency: 0.0
44100 44100 0 44099
event duration: 11025, frequency: 406.37466930385904
55125 55125 0 55124
event duration: 16537, frequency: 456.1401436878537
71662 71662 0 71661
event duration: 11025, frequency: 383.56661168043047
82687 82687 0 82686
event duration: 11025, frequency: 406.37466930385904
93712 93712 0 93711
event duration: 5512, frequency: 0.0
99224 99224 0 99223
event duration: 5512, frequency: 406.37466930385904
104736 104736 0 104735
event duration: 11025, frequency: 456.1401436878537
115761 115761 0 115760
event duration: 11025, frequency: 512.0
126786 126786 0 126785
event duration: 11025, frequency: 512.0
137811 137811 0 137810
event duration: -110250, frequency: 456.1401436878537
137811 137811 0 137810
event duration: 11025, frequency: 256.0